In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

test_data = pd.read_csv(r"D:\Downloads\Germany_Volatile_Test_Data.csv")
predictions = pd.read_csv(r"D:\Downloads\kalman_predictions.csv")

test_data['Datetime'] = pd.to_datetime(test_data['Datetime'])
predictions['Datetime'] = pd.to_datetime(predictions['Datetime'])

data = test_data.merge(predictions, on='Datetime', suffixes=('_actual', '_predicted'))

data.rename(columns={'Price (EUR/MWHE)': 'market_price', 'Predicted Price': 'market_price_predicted'}, inplace=True)

data['market_price'] /= 1000
data['market_price_predicted'] /= 1000

alpha = 0.25
gamma = 0.9
epsilon = 1
epsilon_decay = 0.95
min_epsilon = 0.1
episodes = 1460

battery_capacity = 55
starting_soc = 50
initial_budget = 100
charge_efficiency = 0.95
discharge_efficiency = 0.95

actions = ["Charge", "Discharge", "Hold"]
n_soc_states = 10
n_price_states = 10

def get_state(soc, price):
    soc_state = min(n_soc_states - 1, int(soc / (100 / n_soc_states)))
    price_state = min(n_price_states - 1, int(price / (data['market_price'].max() / n_price_states)))
    return soc_state, price_state

start_time = time.time()
q_table = np.zeros((n_soc_states, n_price_states, len(actions)))

for episode in range(episodes):
    soc = starting_soc
    budget = initial_budget
    for t in range(len(data) - 1):
        current_price = data['market_price'].iloc[t]
        predicted_future_price = data['market_price_predicted'].iloc[t + 1]
        
        state = get_state(soc, current_price)
        
        if np.random.uniform(0, 1) < epsilon:
            action_idx = np.random.choice(len(actions))
        else:
            action_idx = np.argmax(q_table[state])
        action = actions[action_idx]
        
        reward = 0
        soc_kwh = (soc / 100) * battery_capacity    
        if action == "Charge" and soc < 100:
            amount = min(battery_capacity - soc_kwh, 10)
            cost = current_price * amount * charge_efficiency
            reward = ((predicted_future_price * amount) - cost) * 0.1
            soc += (amount / battery_capacity) * 100
        elif action == "Discharge" and soc > 0:
            amount = min(soc_kwh, 10)
            revenue = predicted_future_price * amount * discharge_efficiency
            reward = ((current_price * amount)-revenue) * 0.1
            soc -= (amount / battery_capacity) * 100
        elif action == "Hold":
            price_ratio = predicted_future_price / current_price
            if 0.95 < price_ratio < 1.05:
                reward = 0.1 * current_price
            else:
                reward = -0.1 * current_price

        next_state = get_state(soc, predicted_future_price)
        best_future_q = np.max(q_table[next_state])
        q_table[state][action_idx] += alpha * (reward + gamma * best_future_q - q_table[state][action_idx])
    
    epsilon = max(min_epsilon, epsilon * epsilon_decay)

charge_indices, discharge_indices = [], []
soc = starting_soc
budget = initial_budget
cumulative_profits, soc_values = [], [soc]
charge_total, discharge_total = 0, 0

for t in range(len(data) - 1):
    current_price = data['market_price'].iloc[t]
    predicted_future_price = data['market_price_predicted'].iloc[t + 1]
    
    state = get_state(soc, current_price)
    action_idx = np.argmax(q_table[state])
    action = actions[action_idx]
    
    soc_kwh = (soc / 100) * battery_capacity
    if action == "Charge" and soc < 100:
        amount = min(battery_capacity - soc_kwh, 10)
        budget -= current_price * amount
        soc += (amount / battery_capacity) * 100
        charge_total += amount
        charge_indices.append(t)
    elif action == "Discharge" and soc > 0:
        amount = min(soc_kwh, 10)
        budget += predicted_future_price * amount * discharge_efficiency
        soc -= (amount / battery_capacity) * 100
        discharge_total += amount
        discharge_indices.append(t)
    
    soc = max(0, min(100, soc))
    cumulative_profits.append(budget - initial_budget)
    soc_values.append(soc)

elapsed_time = time.time() - start_time

plt.figure(figsize=(12, 6))
plt.plot(cumulative_profits, label='Cumulative Profit (EUR)')
plt.xlabel('Time Step')
plt.ylabel('Profit (EUR)')
plt.legend()
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(soc_values, label='State of Charge (%)', color='g')
plt.xlabel('Time Step')
plt.ylabel('SOC (%)')
plt.legend()
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(data.index[:-1], data['market_price'][:-1], label='Market Price', color='blue')
plt.scatter(charge_indices, data['market_price'].iloc[charge_indices], color='green', label='Charge', marker='^')
plt.scatter(discharge_indices, data['market_price'].iloc[discharge_indices], color='red', label='Discharge', marker='v')
plt.xlabel('Time Step')
plt.ylabel('Price (EUR/kWh)')
plt.legend()
plt.show()

print(f"Money Earned (EUR): {budget - initial_budget}")
print(f"Mean DoD (kWh): {discharge_total / len(data)}")
print(f"Mean DoC (kWh): {charge_total / len(data)}")
print(f"Computational Time (s): {elapsed_time}")
